 # Pathway Commons Pathways

 Author: Moshe Silverstein
 Date: 08-18
 Data Source Home: https://www.pathwaycommons.org/
 Data Source Download: http://www.pathwaycommons.org/archives/PC2/v12/

 Reviewer: Charles Dai <br>
 Updated: 6-20

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome-helper.utility_functions as uf
import harmonizome-helper.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

 ### Python Version

In [ ]:
sys.version

 # Initialization

 ### Choose Attribute

In [ ]:
%%appyter code_eval

{% set attribute = ChoiceField(
    name='attribute',
    label='Attribute',
    choices={
        'Pathways': "'Pathways'",
        'Protein-Protein Interactions': "'PPI'"
    },
    default='Protein-Protein Interactions',
    section='data'
) %}

 ### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

 ### Output Path

In [ ]:
%%appyter code_exec

output_name = 'pathway_commons_' + {{attribute}}.lower()

path = 'Output/Pathway-Commons-' + {{attribute}}
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Pathway Commons Database',
) %}

 # Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.txt$',
    name='phenotype_gene_list', 
    label='Phenotypes to Genes', 
    default='Input/Pathway-Commons/PathwayCommons12.All.hgnc.txt',
    section='data')
}}, sep='\t', usecols=['PARTICIPANT_A', 'PARTICIPANT_B', 'PATHWAY_NAMES'])

In [ ]:
df.head()

In [ ]:
df.shape

 # Pre-process Data

 ## Get Relevant Data

In [ ]:
%%appyter code_exec
if {{attribute}} == 'Pathways':
    # Put both participants in pathway in single list, some participant b
    # have more than one name, so take first name
    df2 = df[['PARTICIPANT_B', 'PATHWAY_NAMES']]
    df2['PARTICIPANT_B'] = df2['PARTICIPANT_B'].map(lambda x: x.split(';')[0])
    df2.columns=['PARTICIPANT_A', 'PATHWAY_NAMES']
    df = pd.concat([df[['PARTICIPANT_A', 'PATHWAY_NAMES']], df2])
elif {{attribute}} == 'PPI':
    df = df[['PARTICIPANT_A', 'PARTICIPANT_B']]
df = df.dropna()
df = df.set_index('PARTICIPANT_A')

In [ ]:
df.head()

 # Filter Data

 ## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
%%appyter code_exec
if {{attribute}} == 'PPI':
    df = uf.mapgenesymbols(df.reset_index().set_index('PARTICIPANT_B'), 
                            symbol_lookup)
    df = df.reset_index().set_index('PARTICIPANT_A')
df = uf.mapgenesymbols(df, symbol_lookup)

In [ ]:
df.shape

 # Analyze Data

 ## Create Binary Matrix

In [ ]:
binary_matrix = uf.createBinaryMatrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.saveData(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

 ## Create Gene List

In [ ]:
gene_list = uf.createGeneList(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.saveData(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

 ## Create Attribute List

In [ ]:
attribute_list = uf.createAttributeList(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.saveData(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

 ## Create Gene and Attribute Set Libraries

In [ ]:
uf.createUpGeneSetLib(binary_matrix, path, output_name + '_gene_up_set')

In [ ]:
uf.createUpAttributeSetLib(binary_matrix, path, 
                           output_name + '_attribute_up_set')

 ## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.saveData(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

 ## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.saveData(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

 ## Create Gene-Attribute Edge List

In [ ]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, 
                               path, output_name + '_gene_attribute_edge_list')

 # Create Downloadable Save File

In [ ]:
uf.createArchive(path)

 ### Link to download output files: [click here](./output_archive.zip)